<a href="https://colab.research.google.com/github/Nishanth-nishu/Algerian_forest_fire_prediction/blob/main/OpenVoice_colab.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# Clone the OpenVoice repository
!git clone https://github.com/myshell-ai/OpenVoice.git
%cd OpenVoice

# Install dependencies
!pip install -e .
!pip install torch torchaudio numpy scipy soundfile gradio
!pip install git+https://github.com/myshell-ai/MeloTTS.git
!python -m unidic download  # Required for multi-lingual text processing


Cloning into 'OpenVoice'...
remote: Enumerating objects: 453, done.
remote: Counting objects: 100% (6/6), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 453 (delta 4), reused 3 (delta 3), pack-reused 447 (from 2)
Receiving objects: 100% (453/453), 3.85 MiB | 12.77 MiB/s, done.
Resolving deltas: 100% (214/214), done.
/content/OpenVoice
Obtaining file:///content/OpenVoice
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 35.3 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.8/2.8 MB 69.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 69.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.1/48.1 kB 3.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━

  Cloning https://github.com/myshell-ai/MeloTTS.git to /tmp/pip-req-build-4hugu_ri
  Running command git clone --filter=blob:none --quiet https://github.com/myshell-ai/MeloTTS.git /tmp/pip-req-build-4hugu_ri
  Resolved https://github.com/myshell-ai/MeloTTS.git to commit 209145371cff8fc3bd60d7be902ea69cbdb7965a
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 106.7/106.7 kB 3.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.4/47.4 MB 14.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.5/73.5 kB 4.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 180.3/180.3 kB 13.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.6/101.6 kB 8.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ..

In [1]:
from IPython.display import HTML, display, clear_output
import ipywidgets as widgets
from google.colab import files
import os

class VideoDubbingUI:
    def __init__(self):
        self.upload_path = None
        self.create_ui()

    def create_ui(self):
        # Title
        display(HTML("<h2>Japanese-to-English Video Dubbing System</h2>"))

        # File Upload Section
        self.upload_button = widgets.Button(
            description='Upload Video',
            button_style='primary',
            icon='upload'
        )
        self.upload_button.on_click(self.handle_upload)
        self.file_label = widgets.Label('No file selected')

        # Language Selection (for future extensibility)
        self.source_lang = widgets.Dropdown(
            options=['Japanese'],
            value='Japanese',
            description='Source:',
            disabled=True
        )

        self.target_lang = widgets.Dropdown(
            options=['English'],
            value='English',
            description='Target:',
            disabled=True
        )

        # Process Control
        self.start_button = widgets.Button(
            description='Start Process',
            button_style='success',
            disabled=True
        )
        self.start_button.on_click(self.start_process)

        self.pause_button = widgets.Button(
            description='Pause',
            button_style='warning',
            disabled=True
        )
        self.pause_button.on_click(self.pause_process)

        self.stop_button = widgets.Button(
            description='Stop',
            button_style='danger',
            disabled=True
        )
        self.stop_button.on_click(self.stop_process)

        # Progress Bar
        self.progress = widgets.FloatProgress(
            value=0,
            min=0,
            max=100,
            description='Progress:',
            bar_style='info',
            orientation='horizontal'
        )

        # Status Message
        self.status = widgets.HTML(value="System Ready")

        # Preview Section (placeholder)
        self.preview = widgets.HTML(value='<div style="background-color: #f0f0f0; padding: 10px; text-align: center;">Video preview will appear here</div>')

        # Feedback Section
        self.translation_rating = widgets.IntSlider(
            value=5,
            min=1,
            max=10,
            description='Translation:',
            disabled=True
        )

        self.voice_rating = widgets.IntSlider(
            value=5,
            min=1,
            max=10,
            description='Voice:',
            disabled=True
        )

        self.comments = widgets.Textarea(
            placeholder='Additional comments...',
            disabled=True
        )

        self.feedback_button = widgets.Button(
            description='Submit Feedback',
            button_style='info',
            disabled=True
        )
        self.feedback_button.on_click(self.submit_feedback)

        # Download Section
        self.download_button = widgets.Button(
            description='Download Result',
            button_style='success',
            disabled=True
        )
        self.download_button.on_click(self.download_result)

        # Layout
        upload_box = widgets.VBox([self.upload_button, self.file_label])
        lang_box = widgets.HBox([self.source_lang, self.target_lang])
        control_box = widgets.HBox([self.start_button, self.pause_button, self.stop_button])
        progress_box = widgets.VBox([self.progress, self.status])
        feedback_box = widgets.VBox([
            self.translation_rating,
            self.voice_rating,
            self.comments,
            self.feedback_button
        ])

        main_layout = widgets.VBox([
            upload_box,
            lang_box,
            control_box,
            progress_box,
            self.preview,
            feedback_box,
            self.download_button
        ])

        display(main_layout)

    def handle_upload(self, button):
        """Handle video file upload"""
        try:
            uploaded = files.upload()
            if uploaded:
                filename = list(uploaded.keys())[0]
                self.upload_path = filename
                self.file_label.value = f'Uploaded: {filename}'
                self.start_button.disabled = False
                self.status.value = "Video uploaded successfully"
        except Exception as e:
            self.status.value = f"Upload error: {str(e)}"

    def start_process(self, button):
        """Start the dubbing process"""
        self.progress.value = 0
        self.status.value = "Processing started..."
        self.start_button.disabled = True
        self.pause_button.disabled = False
        self.stop_button.disabled = False

        # Simulate processing steps (replace with actual processing)
        import time
        for i in range(5):
            time.sleep(1)
            self.progress.value += 20
            steps = ["Speech recognition", "Translation", "Voice synthesis",
                    "Lip sync", "Final processing"]
            self.status.value = f"Step {i+1}/5: {steps[i]}"

        self.status.value = "Processing complete!"
        self.enable_feedback()

    def pause_process(self, button):
        """Pause the current process"""
        self.status.value = "Process paused"
        self.start_button.disabled = False

    def stop_process(self, button):
        """Stop the current process"""
        self.progress.value = 0
        self.status.value = "Process stopped"
        self.start_button.disabled = False
        self.pause_button.disabled = True
        self.stop_button.disabled = True

    def enable_feedback(self):
        """Enable feedback controls after processing"""
        self.translation_rating.disabled = False
        self.voice_rating.disabled = False
        self.comments.disabled = False
        self.feedback_button.disabled = False
        self.download_button.disabled = False

    def submit_feedback(self, button):
        """Handle feedback submission"""
        feedback = {
            'translation_rating': self.translation_rating.value,
            'voice_rating': self.voice_rating.value,
            'comments': self.comments.value
        }
        self.status.value = "Thank you for your feedback!"

    def download_result(self, button):
        """Handle result download"""
        if self.upload_path:
            # In a real implementation, this would download the processed video
            self.status.value = "Download started..."

# Create and display the UI
dubbing_ui = VideoDubbingUI()

In [ ]:
# First cell - Install dependencies
%%capture
!apt-get update && apt-get install -y ffmpeg
!pip install moviepy pysrt deep_translator pydub
!git clone https://github.com/myshell-ai/OpenVoice
!git clone https://github.com/myshell-ai/MeloTTS
!pip install git+https://github.com/myshell-ai/MeloTTS.git
!python -m unidic download
!mv OpenVoice/checkpoints .
!pip install torch==2.0.1 fairseq soundfile praat-parselmouth pytest-shutil torchcrepe
!pip install -e OpenVoice

In [5]:
# First cell - Install dependencies
%%capture
!apt-get update && apt-get install -y ffmpeg
!pip install moviepy pysrt deep_translator pydub
!git clone https://github.com/myshell-ai/OpenVoice
!git clone https://github.com/myshell-ai/MeloTTS
!pip install git+https://github.com/myshell-ai/MeloTTS.git
!python -m unidic download
!mv OpenVoice/checkpoints .
!pip install torch==2.0.1 fairseq soundfile praat-parselmouth pytest-shutil torchcrepe
!pip install -e OpenVoice

# Second cell - Import required libraries
import os
import torch
import moviepy.editor as mp
import pysrt
from deep_translator import GoogleTranslator
from openvoice import se_extractor
from openvoice.api import ToneColorConverter
from pydub import AudioSegment
import tempfile
from melotts import FastSpeech2TTSP
import numpy as np
import soundfile as sf

# Third cell - Initialize OpenVoice and MeloTTS
def initialize_openvoice():
    ckpt_converter = 'checkpoints/converter'
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    output_dir = 'outputs'
    os.makedirs(output_dir, exist_ok=True)

    tone_color_converter = ToneColorConverter(f'{ckpt_converter}/config.json', device=device)
    tone_color_converter.load_ckpt(f'{ckpt_converter}/checkpoint.pth')

    return tone_color_converter, output_dir, device

# Initialize MeloTTS for different languages
def setup_melotts(device):
    tts_models = {
        'en': FastSpeech2TTSP.from_pretrained('melotts/fastspeech2-en-ljspeech', device=device),
        'es': FastSpeech2TTSP.from_pretrained('melotts/fastspeech2-es-css10', device=device),
        'fr': FastSpeech2TTSP.from_pretrained('melotts/fastspeech2-fr-css10', device=device),
        'de': FastSpeech2TTSP.from_pretrained('melotts/fastspeech2-de-css10', device=device),
        'zh': FastSpeech2TTSP.from_pretrained('melotts/fastspeech2-zh-aishell3', device=device),
        'ja': FastSpeech2TTSP.from_pretrained('melotts/fastspeech2-ja-jsut', device=device),
        'ko': FastSpeech2TTSP.from_pretrained('melotts/fastspeech2-ko-kss', device=device)
    }
    return tts_models

# Fourth cell - Translation and audio generation functions
def translate_srt(input_srt, target_languages):
    try:
        subs = pysrt.open(input_srt)

        for lang in target_languages:
            translator = GoogleTranslator(source="en", target=lang)
            translated_subs = subs[:]

            for sub in translated_subs:
                sub.text = translator.translate(sub.text)

            output_srt = f"subtitles_{lang}.srt"
            translated_subs.save(output_srt, encoding="utf-8")
            print(f"Translated subtitles saved to: {output_srt}")

        return True
    except Exception as e:
        print(f"Error during translation: {e}")
        return False

def generate_dubbed_audio(srt_file, output_audio, lang, tone_color_converter, source_se, target_se, tts_model):
    subs = pysrt.open(srt_file)
    audio_segments = []
    last_end_time = 0

    for sub in subs:
        text = sub.text.replace("\n", " ")

        # Generate base audio using MeloTTS
        temp_wav = tempfile.NamedTemporaryFile(suffix='.wav', delete=False)
        temp_output = tempfile.NamedTemporaryFile(suffix='.wav', delete=False)

        # Generate speech using MeloTTS
        audio_array = tts_model.synthesize(text)
        sf.write(temp_wav.name, audio_array, tts_model.sampling_rate)

        # Convert tone color
        tone_color_converter.convert(
            audio_src_path=temp_wav.name,
            src_se=source_se,
            tgt_se=target_se,
            output_path=temp_output.name,
            message="@MyShell"
        )

        # Load and process audio segment
        audio_clip = AudioSegment.from_wav(temp_output.name)
        subtitle_start = sub.start.ordinal / 1000.0
        subtitle_end = sub.end.ordinal / 1000.0

        if subtitle_start > last_end_time:
            gap_duration = (subtitle_start - last_end_time) * 1000
            silence = AudioSegment.silent(duration=gap_duration)
            audio_segments.append(silence)

        audio_clip = audio_clip.set_frame_rate(22050).set_channels(1)
        audio_clip = audio_clip[:int((subtitle_end - subtitle_start) * 1000)]

        audio_segments.append(audio_clip)
        last_end_time = subtitle_end

        # Clean up temporary files
        os.unlink(temp_wav.name)
        os.unlink(temp_output.name)

    final_audio = sum(audio_segments)
    final_audio.export(output_audio, format="mp3")
    print(f"Generated dubbed audio for language {lang}")

# Fifth cell - Video processing function
def merge_subtitles_audio_video(video_path, audio_path, subtitles_path, output_path):
    try:
        video = mp.VideoFileClip(video_path)
        audio = mp.AudioFileClip(audio_path)

        if audio.duration > video.duration:
            audio = audio.subclip(0, video.duration)
        elif audio.duration < video.duration:
            audio = audio.fx(mp.vfx.loop, duration=video.duration)

        subs = pysrt.open(subtitles_path)
        subtitle_clips = []

        for sub in subs:
            subtitle = mp.TextClip(
                sub.text,
                fontsize=24,
                color='white',
                font='DejaVuSans',
                method='caption',
                size=(video.w, None)
            ).set_start(
                sub.start.ordinal / 1000.0
            ).set_duration(
                (sub.end.ordinal - sub.start.ordinal) / 1000.0
            ).set_pos(('center', 'bottom'))

            subtitle_clips.append(subtitle)

        final_video = video.set_audio(audio)
        final_video = mp.CompositeVideoClip([final_video, *subtitle_clips])

        final_video.write_videofile(
            output_path,
            codec='libx264',
            audio_codec='aac'
        )

        print(f"Video with subtitles and dubbed audio created successfully: {output_path}")

    except Exception as e:
        print(f"An error occurred: {e}")

# Sixth cell - Main processing function
def process_multiple_languages(video_file, reference_speaker, subtitles_file_template, output_file_template, languages):
    # Initialize OpenVoice and MeloTTS
    tone_color_converter, output_dir, device = initialize_openvoice()
    tts_models = setup_melotts(device)

    # Extract reference speaker embedding
    source_se, _ = se_extractor.get_se(reference_speaker, tone_color_converter, vad=True)
    target_se = source_se  # For voice cloning, we use the same embedding

    for lang in languages:
        print(f"Processing language: {lang}")

        if lang not in tts_models:
            print(f"Skipping unsupported language: {lang}")
            continue

        # Construct file paths
        dubbed_audio_file = f"{output_dir}/audio_{lang}_synced.mp3"
        subtitles_file = subtitles_file_template.format(lang=lang)
        output_video_file = output_file_template.format(lang=lang)

        # Generate dubbed audio
        generate_dubbed_audio(
            subtitles_file,
            dubbed_audio_file,
            lang,
            tone_color_converter,
            source_se,
            target_se,
            tts_models[lang]
        )

        # Merge everything together
        merge_subtitles_audio_video(
            video_file,
            dubbed_audio_file,
            subtitles_file,
            output_video_file
        )

# Seventh cell - Run the process
# Upload your video and reference audio files to Colab first
video_file = "your_video.mp4"  # Replace with your video file path
reference_speaker = "your_reference.mp3"  # Replace with your reference audio file path
languages = ["en", "es", "fr", "de", "ja", "zh", "ko"]  # Supported languages

subtitles_file_template = "subtitles_{lang}.srt"
output_file_template = "{lang}_final_video_with_subtitles.mp4"

process_multiple_languages(
    video_file,
    reference_speaker,
    subtitles_file_template,
    output_file_template,
    languages
)

ModuleNotFoundError: No module named 'melotts'

In [2]:
# First cell - Run these installation commands
%%capture
!apt-get update && apt-get install -y ffmpeg
!pip install moviepy pysrt deep_translator pydub

# Clone repositories
!git clone https://github.com/myshell-ai/OpenVoice
!git clone https://github.com/myshell-ai/MeloTTS

# Install MeloTTS
!pip install git+https://github.com/myshell-ai/MeloTTS.git
!python -m unidic download

# Install OpenVoice dependencies
!pip install torch==2.0.1
!pip install fairseq
!pip install soundfile
!pip install praat-parselmouth
!pip install pytest-shutil
!pip install torchcrepe

# Download OpenVoice checkpoints
!wget https://myshell-public-repo-hosting.s3.amazonaws.com/checkpoints_v2.tar
!tar -xf checkpoints_v2.tar

# Install OpenVoice in editable mode
import sys
sys.path.append('/content/OpenVoice')

# Second cell - Verify installation
import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

# Print versions to verify
print("OpenVoice installed successfully!")
print(f"Torch version: {torch.__version__}")
print(f"CUDA available: {torch.cuda.is_available()}")

In [34]:
!git clone https://github.com/myshell-ai/MeloTTS.git
!cd MeloTTS
!pip install -e .
!python -m unidic download

Cloning into 'MeloTTS'...
remote: Enumerating objects: 432, done.
remote: Counting objects: 100% (255/255), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 432 (delta 200), reused 158 (delta 158), pack-reused 177 (from 2)
Receiving objects: 100% (432/432), 6.05 MiB | 18.00 MiB/s, done.
Resolving deltas: 100% (225/225), done.
Obtaining file:///content/MeloTTS/MeloTTS
  Preparing metadata (setup.py) ... done
  Attempting uninstall: melotts
    Found existing installation: melotts 0.1.2
    Uninstalling melotts-0.1.2:
      Successfully uninstalled melotts-0.1.2
  Running setup.py develop for melotts
download url: https://cotonoha-dic.s3-ap-northeast-1.amazonaws.com/unidic-3.1.0.zip
Dictionary version: 3.1.0+2021-08-31
unidic-3.1.0.zip: 100% 526M/526M [00:44<00:00, 11.7MB/s]
Finished download.
Downloaded UniDic v3.1.0+2021-08-31 to /usr/local/lib/python3.11/dist-packages/unidic/dicdir


In [32]:
!git clone https://github.com/myshell-ai/MeloTTS.git
%cd MeloTTS
!docker build -t melotts .

Cloning into 'MeloTTS'...
remote: Enumerating objects: 432, done.
remote: Counting objects: 100% (255/255), done.
remote: Compressing objects: 100% (97/97), done.
remote: Total 432 (delta 200), reused 158 (delta 158), pack-reused 177 (from 2)
Receiving objects: 100% (432/432), 6.05 MiB | 15.71 MiB/s, done.
Resolving deltas: 100% (225/225), done.
/content/MeloTTS/MeloTTS
/bin/bash: line 1: docker: command not found


In [38]:
!pip install --upgrade pip
!pip install -e .


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 18.2 MB/s eta 0:00:00
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
Obtaining file:///content/MeloTTS/MeloTTS/MeloTTS
  Preparing metadata (setup.py) ... done
  Attempting uninstall: melotts
    Found existing installation: melotts 0.1.2
    Uninstalling melotts-0.1.2:
      Successfully uninstalled melotts-0.1.2
  DEPRECATION: Legacy editable install of melotts==0.1.2 from file:///content/MeloTTS/MeloTTS/MeloTTS (setup.py develop) is deprecated. pip 25.1 will enforce this behaviour change. A possible replacement is to add a pyproject.toml or enable --use-pep517, and use setuptools >= 64. If the resulting installation is not behaving as expected, try using --config-settings editable_mode=compat. Please consult the setuptools documentation for more information. Discussion can be found at https://github.com/pypa/pip/issues/11457
  Running

In [6]:
%%capture
!apt-get update && apt-get install -y ffmpeg
!pip install moviepy pysrt deep_translator pydub
!git clone https://github.com/myshell-ai/MeloTTS.git
!cd MeloTTS && pip install -e .
!python -m unidic download

In [7]:
!dir

MeloTTS  sample_data


In [8]:
%cd MeloTTS
!pip install -r requirements.txt


/content/MeloTTS


In [16]:
import nltk
import nltk
nltk.data.path.append('/root/nltk_data')
nltk.download('averaged_perceptron_tagger')

from melo.api import TTS

# Speed is adjustable
speed = 1.0

# CPU is sufficient for real-time inference.
device = 'cpu'  # Will use CPU for inference

# English text
text = "Did you ever hear a folk tale about a giant turtle?"

# Load the model
model = TTS(language='EN', device=device)

# Get speaker IDs
speaker_ids = model.hps.data.spk2id

# Generate audio for different accents
output_path = 'en-us.wav'
model.tts_to_file(text, speaker_ids['EN-US'], output_path, speed=speed)

output_path = 'en-br.wav'
model.tts_to_file(text, speaker_ids['EN-BR'], output_path, speed=speed)

output_path = 'en-india.wav'
model.tts_to_file(text, speaker_ids['EN_INDIA'], output_path, speed=speed)

output_path = 'en-au.wav'
model.tts_to_file(text, speaker_ids['EN-AU'], output_path, speed=speed)

output_path = 'en-default.wav'
model.tts_to_file(text, speaker_ids['EN-Default'], output_path, speed=speed)


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


 > Text split to sentences.
Did you ever hear a folk tale about a giant turtle?
 > ===========================


  0%|          | 0/1 [00:00<?, ?it/s]WARNING:py.warnings:/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:795: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(

  warnings.warn(



model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 1/1 [00:50<00:00, 50.03s/it]


...

100%|██████████| 1/1 [00:39<00:00, 39.94s/it]


 > Text split to sentences.
Did you ever hear a folk tale about a giant turtle?
 > ===========================


100%|██████████| 1/1 [00:06<00:00,  6.94s/it]


 > Text split to sentences.
Did you ever hear a folk tale about a giant turtle?
 > ===========================


100%|██████████| 1/1 [00:35<00:00, 35.01s/it]


In [15]:
import nltk
try:
    nltk.data.find('taggers/averaged_perceptron_tagger')
    print("Resource is available.")
except LookupError:
    print("Resource not found.")


Resource is available.


In [12]:
import nltk
nltk.download('averaged_perceptron_tagger')


[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!


True

In [17]:


# Second cell - Import required libraries
import os
import torch
import moviepy.editor as mp
import pysrt
from deep_translator import GoogleTranslator
from pydub import AudioSegment
import tempfile
import soundfile as sf
from melo.api import TTS
from google.colab import files
import IPython.display as ipd
from IPython.display import HTML

# Third cell - Video Translator Class
class VideoTranslator:
    def __init__(self, device='auto'):
        """Initialize the video translator with TTS models."""
        self.device = 'cuda:0' if torch.cuda.is_available() and device == 'auto' else device

        print("Initializing TTS models...")
        self.ja_tts = TTS(language='JA', device=self.device)
        self.en_tts = TTS(language='EN', device=self.device)

        # Get speaker IDs
        self.en_speaker = self.en_tts.hps.data.spk2id['EN-US']  # Using US English accent

        # Create output directory
        self.output_dir = 'outputs'
        os.makedirs(self.output_dir, exist_ok=True)
        print("Initialization complete!")

    def translate_subtitles(self, input_srt, source_lang='ja', target_lang='en'):
        """Translate subtitles between Japanese and English."""
        try:
            subs = pysrt.open(input_srt)
            translator = GoogleTranslator(source=source_lang, target=target_lang)
            translated_subs = subs[:]

            print("Translating subtitles...")
            for sub in translated_subs:
                sub.text = translator.translate(sub.text)

            output_srt = os.path.join(self.output_dir, f"subtitles_{target_lang}.srt")
            translated_subs.save(output_srt, encoding="utf-8")
            print(f"Translated subtitles saved to: {output_srt}")
            return output_srt

        except Exception as e:
            print(f"Error during translation: {e}")
            return None

    def generate_audio(self, srt_file, output_audio, is_japanese=True, speed=1.0):
        """Generate dubbed audio from subtitles."""
        subs = pysrt.open(srt_file)
        audio_segments = []
        last_end_time = 0

        print("Generating audio...")
        total_subs = len(subs)

        for idx, sub in enumerate(subs, 1):
            print(f"\rProcessing subtitle {idx}/{total_subs}", end="")

            text = sub.text.replace("\n", " ")

            # Generate speech using appropriate TTS model
            tts_model = self.ja_tts if is_japanese else self.en_tts
            speaker_id = None if is_japanese else self.en_speaker

            # Create temporary WAV file
            temp_wav = tempfile.NamedTemporaryFile(suffix='.wav', delete=False)

            try:
                # Generate speech
                if speaker_id is not None:
                    tts_model.tts_to_file(text, speaker_id, temp_wav.name, speed=speed)
                else:
                    tts_model.tts_to_file(text, speaker_id=0, file_path=temp_wav.name, speed=speed)

                # Load and process audio segment
                audio_clip = AudioSegment.from_wav(temp_wav.name)
                subtitle_start = sub.start.ordinal / 1000.0
                subtitle_end = sub.end.ordinal / 1000.0

                if subtitle_start > last_end_time:
                    gap_duration = (subtitle_start - last_end_time) * 1000
                    silence = AudioSegment.silent(duration=gap_duration)
                    audio_segments.append(silence)

                target_duration = (subtitle_end - subtitle_start) * 1000
                if len(audio_clip) > target_duration:
                    audio_clip = audio_clip[:int(target_duration)]

                audio_segments.append(audio_clip)
                last_end_time = subtitle_end

            finally:
                os.unlink(temp_wav.name)

        print("\nMerging audio segments...")
        final_audio = sum(audio_segments)
        final_audio.export(output_audio, format="mp3")
        print(f"Generated dubbed audio saved to: {output_audio}")

    def create_video_with_subtitles(self, video_path, audio_path, subtitles_path, output_path):
        """Merge video with dubbed audio and subtitles."""
        try:
            print("Loading video...")
            video = mp.VideoFileClip(video_path)
            audio = mp.AudioFileClip(audio_path)

            if audio.duration > video.duration:
                audio = audio.subclip(0, video.duration)
            elif audio.duration < video.duration:
                audio = audio.fx(mp.vfx.loop, duration=video.duration)

            print("Adding subtitles...")
            subs = pysrt.open(subtitles_path)
            subtitle_clips = []

            for sub in subs:
                subtitle = mp.TextClip(
                    sub.text,
                    fontsize=24,
                    color='white',
                    font='DejaVuSans',
                    method='caption',
                    size=(video.w, None)
                ).set_start(
                    sub.start.ordinal / 1000.0
                ).set_duration(
                    (sub.end.ordinal - sub.start.ordinal) / 1000.0
                ).set_pos(('center', 'bottom'))

                subtitle_clips.append(subtitle)

            print("Rendering final video...")
            final_video = video.set_audio(audio)
            final_video = mp.CompositeVideoClip([final_video, *subtitle_clips])

            final_video.write_videofile(
                output_path,
                codec='libx264',
                audio_codec='aac'
            )

            print(f"Final video created: {output_path}")

        except Exception as e:
            print(f"Error creating video: {e}")

# Fourth cell - File upload handling and processing function
def process_video_in_colab():
    """Handle file uploads and process video in Colab environment."""
    print("Please upload your video file...")
    video_file = files.upload()
    video_path = list(video_file.keys())[0]

    print("\nPlease upload your Japanese SRT subtitle file...")
    srt_file = files.upload()
    srt_path = list(srt_file.keys())[0]

    # Create output path
    output_path = "translated_video.mp4"

    # Initialize translator
    translator = VideoTranslator()

    # Create output directory
    os.makedirs('outputs', exist_ok=True)

    # Translate subtitles
    print("\nTranslating subtitles...")
    en_subs_path = translator.translate_subtitles(srt_path)
    if not en_subs_path:
        return

    # Generate English audio
    print("\nGenerating English audio...")
    en_audio_path = "outputs/english_audio.mp3"
    translator.generate_audio(en_subs_path, en_audio_path, is_japanese=False, speed=1.0)

    # Create final video
    print("\nCreating final video...")
    translator.create_video_with_subtitles(video_path, en_audio_path, en_subs_path, output_path)

    # Provide download link
    print("\nProcessing complete! The video will be available for download.")
    files.download(output_path)

# Fifth cell - Run the processor
process_video_in_colab()

Please upload your video file...


Saving japnese_final_video_with_subtitles.mp4 to japnese_final_video_with_subtitles.mp4

Please upload your Japanese SRT subtitle file...


IndexError: list index out of range

In [4]:
import os
import torch
from openvoice import se_extractor
from openvoice.api import ToneColorConverter

In [4]:
import os
import torch
import moviepy.editor as mp
import pysrt
from deep_translator import GoogleTranslator
from pydub import AudioSegment
import tempfile
import soundfile as sf
from melo.api import TTS
from google.colab import files
import IPython.display as ipd
from IPython.display import HTML
import whisper  # Added for speech-to-text

class VideoTranslator:
    def __init__(self, device='auto'):
        self.device = 'cuda:0' if torch.cuda.is_available() and device == 'auto' else device
        print("Initializing TTS models...")
        self.ja_tts = TTS(language='JP', device=self.device)
        self.en_tts = TTS(language='EN', device=self.device)
        self.en_speaker = self.en_tts.hps.data.spk2id['EN-US']
        self.output_dir = 'outputs'
        os.makedirs(self.output_dir, exist_ok=True)
        print("Initialization complete!")

    def extract_audio(self, video_path, audio_path="extracted_audio.wav"):
        command = f"ffmpeg -i {video_path} -vn -acodec pcm_s16le -ar 16000 -ac 1 {audio_path}"
        os.system(command)
        return audio_path

    def transcribe_audio(self, audio_path):
        model = whisper.load_model("large")
        result = model.transcribe(audio_path, language="ja")
        return result["segments"]

    def generate_srt_from_transcription(self, transcription_segments, srt_output_path="subtitles_ja.srt"):
        subtitles = []
        for segment in transcription_segments:
            start_time = segment["start"]
            end_time = segment["end"]
            text = segment["text"]
            subtitles.append(pysrt.SubRipItem(index=len(subtitles) + 1, start=start_time, end=end_time, text=text))

        subs = pysrt.SubRipFile()
        subs.extend(subtitles)
        subs.save(srt_output_path, encoding="utf-8")
        print(f"Japanese subtitles saved to {srt_output_path}")
        return srt_output_path

    def translate_subtitles(self, input_srt, source_lang='ja', target_lang='en'):
        try:
            subs = pysrt.open(input_srt)
            translator = GoogleTranslator(source=source_lang, target=target_lang)
            for sub in subs:
                sub.text = translator.translate(sub.text)
            output_srt = os.path.join(self.output_dir, f"subtitles_{target_lang}.srt")
            subs.save(output_srt, encoding="utf-8")
            print(f"Translated subtitles saved to: {output_srt}")
            return output_srt
        except Exception as e:
            print(f"Error during translation: {e}")
            return None

    def generate_audio(self, srt_file, output_audio, is_japanese=True, speed=1.0):
        subs = pysrt.open(srt_file)
        audio_segments = []
        last_end_time = 0
        print("Generating audio...")
        for sub in subs:
            text = sub.text.replace("\n", " ")
            tts_model = self.ja_tts if is_japanese else self.en_tts
            speaker_id = None if is_japanese else self.en_speaker
            temp_wav = tempfile.NamedTemporaryFile(suffix='.wav', delete=False)
            try:
                tts_model.tts_to_file(text, speaker_id, temp_wav.name, speed=speed)
                audio_clip = AudioSegment.from_wav(temp_wav.name)
                subtitle_start = sub.start.ordinal / 1000.0
                subtitle_end = sub.end.ordinal / 1000.0
                if subtitle_start > last_end_time:
                    gap_duration = (subtitle_start - last_end_time) * 1000
                    silence = AudioSegment.silent(duration=gap_duration)
                    audio_segments.append(silence)
                target_duration = (subtitle_end - subtitle_start) * 1000
                if len(audio_clip) > target_duration:
                    audio_clip = audio_clip[:int(target_duration)]
                audio_segments.append(audio_clip)
                last_end_time = subtitle_end
            finally:
                os.unlink(temp_wav.name)
        print("Merging audio segments...")
        final_audio = sum(audio_segments)
        final_audio.export(output_audio, format="mp3")
        print(f"Generated dubbed audio saved to: {output_audio}")

    def create_video_with_subtitles(self, video_path, audio_path, subtitles_path, output_path):
        try:
            print("Loading video...")
            video = mp.VideoFileClip(video_path)
            audio = mp.AudioFileClip(audio_path)
            audio = audio.set_duration(video.duration)
            print("Adding subtitles...")
            subs = pysrt.open(subtitles_path)
            subtitle_clips = []
            for sub in subs:
                subtitle = mp.TextClip(sub.text, fontsize=24, color='white', font='DejaVuSans', method='caption', size=(video.w, None)).set_start(sub.start.ordinal / 1000.0).set_duration((sub.end.ordinal - sub.start.ordinal) / 1000.0).set_pos(('center', 'bottom'))
                subtitle_clips.append(subtitle)
            print("Rendering final video...")
            final_video = video.set_audio(audio)
            final_video = mp.CompositeVideoClip([final_video, *subtitle_clips])
            final_video.write_videofile(output_path, codec='libx264', audio_codec='aac')
            print(f"Final video created: {output_path}")
        except Exception as e:
            print(f"Error creating video: {e}")

def process_video_in_colab():
    print("Please upload your video file...")
    video_file = files.upload()
    video_path = list(video_file.keys())[0]
    translator = VideoTranslator()
    audio_file = translator.extract_audio(video_path)
    transcription_segments = translator.transcribe_audio(audio_file)
    srt_path = translator.generate_srt_from_transcription(transcription_segments)
    en_subs_path = translator.translate_subtitles(srt_path)
    if not en_subs_path:
        return
    en_audio_path = "outputs/english_audio.mp3"
    translator.generate_audio(en_subs_path, en_audio_path, is_japanese=False, speed=1.0)
    output_path = "translated_video.mp4"
    translator.create_video_with_subtitles(video_path, en_audio_path, en_subs_path, output_path)
    print("Processing complete! The video will be available for download.")
    files.download(output_path)

process_video_in_colab()


  warnings.warn(

The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(



Please upload your video file...


Saving japnese_final_video_with_subtitles (1).mp4 to japnese_final_video_with_subtitles (1).mp4
Initializing TTS models...


  WeightNorm.apply(module, name, dim)



Initialization complete!


100%|█████████████████████████████████████| 2.88G/2.88G [00:54<00:00, 56.8MiB/s]
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")



RuntimeError: Failed to load audio: ffmpeg version 4.4.2-0ubuntu0.22.04.1 Copyright (c) 2000-2021 the FFmpeg developers
  built with gcc 11 (Ubuntu 11.2.0-19ubuntu1)
  configuration: --prefix=/usr --extra-version=0ubuntu0.22.04.1 --toolchain=hardened --libdir=/usr/lib/x86_64-linux-gnu --incdir=/usr/include/x86_64-linux-gnu --arch=amd64 --enable-gpl --disable-stripping --enable-gnutls --enable-ladspa --enable-libaom --enable-libass --enable-libbluray --enable-libbs2b --enable-libcaca --enable-libcdio --enable-libcodec2 --enable-libdav1d --enable-libflite --enable-libfontconfig --enable-libfreetype --enable-libfribidi --enable-libgme --enable-libgsm --enable-libjack --enable-libmp3lame --enable-libmysofa --enable-libopenjpeg --enable-libopenmpt --enable-libopus --enable-libpulse --enable-librabbitmq --enable-librubberband --enable-libshine --enable-libsnappy --enable-libsoxr --enable-libspeex --enable-libsrt --enable-libssh --enable-libtheora --enable-libtwolame --enable-libvidstab --ena

In [2]:
import whisper

model = whisper.load_model("small")  # Or use "small" for faster results
result = model.transcribe("extracted_audio.wav", language="ja")

print(result["text"])  # This should print the transcribed Japanese text


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


はい。My name is Lisa Gasquil and I'm coordinator for the 31 short tone, for new graduate teaching assistant like you.i had to prepare 3 minute present for my peers back in 2013.started my program, The purpose of my is to help you prepare for your own 3 minute, fit you to present next month at the, there are 3 main points that I'm going.No.1.PresentationStyle.&No.1.topicofchoice.it is 100% to use or choose something that you are interested in, that you are passionate about.Summing that you feel comfortable sharing students.it could be a hobby.O.I.T.Q.T.Simple.E.B.What.You.A.Stada.SchoolLightNOW.No.2.PresentationStyle.it is in, always remember you are presenting.In front of a group of students, I think they have to feel connected.One of the most important ways is how they are you available to them physically.So I want you to carry yourself in such a way any given time can see you.Sink on 3 way mirror you have the left side and you have the right side, always up to every side of the room.An

In [3]:
import whisper
model = whisper.load_model("small")  # Try "small" to test
print("Model loaded successfully!")


100%|███████████████████████████████████████| 461M/461M [00:09<00:00, 51.3MiB/s]


Model loaded successfully!


In [1]:
!pip install torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118  # Use cu118 if you have CUDA 11.8
!pip install openai-whisper


Looking in indexes: https://download.pytorch.org/whl/cu118


In [1]:
from melo.api import TTS

# Speed is adjustable
speed = 1.0
device = 'cpu' # or cuda:0

text = "彼は毎朝ジョギングをして体を健康に保っています。"
model = TTS(language='JP', device=device)
speaker_ids = model.hps.data.spk2id

output_path = 'jp.wav'
model.tts_to_file(text, speaker_ids['JP'], output_path, speed=speed)

KeyboardInterrupt: 